In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm
import xgboost as xgb

import pickle

In [2]:
data_path = '../data/'
test = pickle.load(open(data_path + 'test', 'r'))
train = pickle.load(open(data_path + 'train', 'r'))

In [3]:
# print train.head(0)

In [5]:
#<class 'pandas.core.series.Series'> object
#pandas.core.frame.DataFrame
print train.head()

                                           msno  \
0  FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=   
1  Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=   
2  Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=   
3  Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=   
4  FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=   

                                        song_id source_system_tab  \
0  BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=           explore   
1  bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=        my library   
2  JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=        my library   
3  2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=        my library   
4  3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=           explore   

    source_screen_name      source_type  target  song_length genre_ids  \
0              Explore  online-playlist       1     206471.0       359   
1  Local playlist more   local-playlist       1     284584.0      1259   
2  Local playlist more   local-playlist       1     2253

In [6]:
# Preprocess dataset
cols = list(train.columns)
cols.remove('target')

for col in tqdm(cols):
    try:
        if train[col].dtype == 'object':
            train[col] = train[col].apply(str)
            test[col] = test[col].apply(str)

            le = LabelEncoder()
            train_vals = list(train[col].unique())
            test_vals = list(test[col].unique())
            le.fit(train_vals + test_vals)
            train[col] = le.transform(train[col])
            test[col] = le.transform(test[col])

            print(col + ': ' + str(len(train_vals)) + ', ' + str(len(test_vals)))
    except:
        print (col, type(train[col]))
        break

  4%|▍         | 1/25 [00:34<13:53, 34.72s/it]

msno: 30755, 25131


  8%|▊         | 2/25 [02:14<25:46, 67.23s/it]

song_id: 359966, 224753


 12%|█▏        | 3/25 [02:33<18:42, 51.00s/it]

...

song_name: 234112, 154708


 52%|█████▏    | 13/25 [07:22<06:48, 34.02s/it]

song_country: 110, 97


100%|██████████| 25/25 [07:36<00:00, 18.27s/it]

gender: 3, 3


In [7]:
print (train.head(2))

    msno  song_id  source_system_tab  source_screen_name  source_type  target  \
0   9176    86884                  2                   8            7       1   
1  19273   260594                  4                   9            5       1   

   song_length  genre_ids  artist_name  composer        ...         bd  \
0     206471.0        308         3785     16654        ...          0   
1     284584.0         98        36868        71        ...         24   

   gender  registered_via  registration_init_time  expiration_date  \
0       0               7                20120102                5   
1       1               9                20110525               11   

   registration_year  registration_month  registration_date  expiration_year  \
0               2012                   1                  2             2017   
1               2011                   5                 25             2017   

   expiration_month  
0                10  
1                 9  

[2 rows x 26 c

In [8]:
X = np.array(train.drop(['target'], axis=1))
y = train['target'].values

X_test = np.array(test.drop(['id'], axis=1))
ids = test['id'].values

# X_train, X_valid, y_train, y_valid = train_test_split(X, y, \
#     test_size=0.2, random_state=0)
l = int(np.shape(X)[0] * 0.8)
X_train, X_valid, y_train, y_valid = X[:l], X[l:], y[:l], y[l:]


d_train = xgb.DMatrix(X_train, label=y_train)
d_valid = xgb.DMatrix(X_valid, label=y_valid) 
d_test = xgb.DMatrix(X_test)

eval_set = [(X_train, y_train), (X_valid, y_valid)]
watchlist = [(d_train, 'train'), (d_valid, 'valid')]

In [9]:
# Train model, evaluate and make predictions
def model(eta, max_depth):
    params = {}
    params['objective'] = 'binary:logistic'
    params['eta'] = eta #0.7
    params['max_depth'] = max_depth #5
    params['silent'] = 1
    params['eval_metric'] = 'auc'
    params['subsample'] = 1.0 #1.0
    params['min_child_weight'] = 5 #5
    params['colsample_bytree'] = 0.2 # 0.2
    evals_result = {}
    
    model = xgb.train(params, d_train, 105, watchlist, early_stopping_rounds=20, \
        maximize=True, verbose_eval=10, evals_result = evals_result)

    p_test = model.predict(d_test)
    return [p_test, [evals_result['train']['auc'][-1], evals_result['valid']['auc'][-1]]]

In [18]:
weights = []
for max_depth in range(10, 15):
    for eta in [0.3]:
        p_test, res = model(eta, max_depth)
        weights += res,

[0]	train-auc:0.662731	valid-auc:0.598634
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 20 rounds.
[10]	train-auc:0.704115	valid-auc:0.626949
[20]	train-auc:0.720329	valid-auc:0.635454
[30]	train-auc:0.734481	valid-auc:0.643341
[40]	train-auc:0.744398	valid-auc:0.646602
[50]	train-auc:0.750529	valid-auc:0.649651
[60]	train-auc:0.758824	valid-auc:0.65314
[70]	train-auc:0.763944	valid-auc:0.654889
[80]	train-auc:0.768342	valid-auc:0.65673
[90]	train-auc:0.772604	valid-auc:0.658156
[100]	train-auc:0.775828	valid-auc:0.658527
[104]	train-auc:0.777442	valid-auc:0.65917
[0]	train-auc:0.663824	valid-auc:0.598275
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 20 rounds.
[10]	train-auc:0.710352	valid-auc:0.626976
[20]	train-auc:0.731513	valid-auc:0.636657
[30]	train-auc:0.748417	valid-auc:0.645228
[40]	train-auc:0.760453	valid

In [19]:
weights = [x[1] for x in weights]
print (weights)
#16-20
#0.665855, 0.664718, 0.664207, 0.664103, 0.661541
##11-15
#0.661144, 0.664255, 0.666004, 0.666707, 0.666783
#15

[0.65917, 0.661144, 0.664255, 0.666004, 0.666707]


In [12]:
total = np.sum(weights)
print (total)
result = (np.array(p_tests).T * np.array(weights)).T

3.987207


In [13]:
print (np.shape(result))
mat = np.sum(np.matrix(result), axis = 0)
p_test = [x / total for x in (mat.tolist()[0])]
print (np.shape(p_test))

(6, 2556790)
(2556790,)


In [14]:
for x in p_test[:10]:
    print (x)

0.4642308979
0.504770238454
0.153423018504
0.143255339305
0.0527307230864
0.0574776432928
0.0489744545509
0.718005915934
0.0675593516035
0.705625235429


In [15]:
# with open('p_tests.pickle', 'wb') as handle:
#     pickle.dump(p_tests, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('weights.pickle', 'wb') as handle:
#     pickle.dump(weights, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [16]:
print (np.sum(p_test))

1134167.91314


In [17]:
# Prepare submission
subm = pd.DataFrame()
subm['id'] = ids
subm['target'] = p_test
print(len(ids), len(p_test))
subm.to_csv('../submissions/submission.csv', index=False)

(2556790, 2556790)
